In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "mistralai/Mistral-7B-v0.1"

# 4bit configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_fast=False,
    trust_remote_code=True
)

# Load model WITHOUT accelerate, WITHOUT device_map
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map=None,           # <- IMPORTANT
)

# Manually move model to GPU
# model.to("cuda:0")

print("Model loaded on:", next(model.parameters()).device)

prompt = "Explain quantum computing in simple terms."
inputs = tokenizer(prompt, return_tensors="pt")

with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7
    )

print(tokenizer.decode(out[0], skip_special_tokens=True))

/root/miniconda/envs/likhitenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Model loaded on: cuda:0


/root/miniconda/envs/likhitenv/lib/python3.10/site-packages/transformers/generation/utils.py:1460: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Explain quantum computing in simple terms.

Quantum computing is a computer technology that uses the properties of quantum mechanics such as superposition and entanglement.

What are the advantages of quantum computing?

Quantum computing has the potential to solve problems that cannot be solved by classical computers. Classical computers encode information as bits (binary digits, either 0 or 1), while quantum computers encode information as qubits (quantum bits, which can be in superposition of 0 and 1). This


In [15]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

In [14]:
num_layers = len(model.model.layers)
print(num_layers)

32


In [16]:
print(model.model.layers[0].self_attn)

MistralSdpaAttention(
  (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
  (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
  (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
  (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
  (rotary_emb): MistralRotaryEmbedding()
)
